In [1]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

model_dir = '/opt/ml/model'
train_instance_type = 'local_gpu'
hyperparameters = {'epochs': 20, 
                   'batch_size': 64
                  }



In [ ]:
local_estimator = TensorFlow(entry_point='train.py',
                       source_dir='code',
                       model_dir=model_dir,
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name='tf-20-newsgroups',
                       framework_version='1.13',
                       py_version='py3',
                       script_mode=True)

In [ ]:
inputs = {'train': f'file://{train_dir}',
          'val': f'file://{val_dir}',
          'embedding': f'file://{embedding_dir}'}




In [ ]:
s3_prefix = 'tf-20-newsgroups'

traindata_s3_prefix = '{}/data/input'.format(s3_prefix)
valdata_s3_prefix = '{}/data/label'.format(s3_prefix)


train_s3 = sagemaker.Session().upload_data(path='./data/input/', key_prefix=traindata_s3_prefix)
val_s3 = sagemaker.Session().upload_data(path='./data/label/', key_prefix=valdata_s3_prefix)


inputs = {'train':train_s3, 'val': val_s3}
print(inputs)

In [ ]:
train_instance_type = 'ml.p3.2xlarge'
hyperparameters = {'epochs': 20, 
                   'batch_size': 128 ,
                   'train_input': "./data/input/" ,
                   'label_input': "./data/label/"
                  }

estimator = TensorFlow(entry_point='train.py',
                       source_dir='code',
                       model_dir=model_dir,
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name='tf-20-newsgroups',
                       framework_version='1.13',
                       py_version='py3',
                       script_mode=True)

In [ ]:
estimator.fit(inputs)

In [ ]:
predictor = TensorFlow(entry_point='predict.py',
                       source_dir='code',
                       model_dir=model_dir,
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name='tf-20-newsgroups',
                       framework_version='1.13',
                       py_version='py3',
                       script_mode=True)

In [ ]:
predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m5.xlarge')

In [ ]:
predictor.predict()